In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import re 
import string
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.chunk import ne_chunk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.tokenize import MWETokenizer
import pickle
import pandas as pd
from sklearn import datasets
import gensim
import pyLDAvis
from pyLDAvis import gensim as gensimvis
import logging
from tqdm import tqdm
from pprint import pprint

In [3]:
listings = pd.read_csv('listings 2.csv')

In [3]:
import pickle 

with open('df_final.pkl','rb') as read_file:
    df_final = pickle.load(read_file)

In [5]:
reviews = df_final.iloc[:,5]

In [6]:
from nltk.tokenize import sent_tokenize
review_sent = []
for i in reviews:
    review_sent += sent_tokenize(str(i))

In [7]:
len(review_sent)

1294255

# Data Cleaning
### Drop duplicates

In [9]:
review_sent = set(review_sent)

In [10]:
len(review_sent)

1153359

### LDA

In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
df_list = list(sent_to_words(review_sent))

In [14]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(df_list, min_count=20) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[df_list], min_count=10)  
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_model = gensim.models.phrases.Phraser(bigram)
trigram_model = gensim.models.phrases.Phraser(trigram)

/Users/zhaoze/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [15]:
clean_sents = [trigram_model[bigram_model[t]] for t in df_list]

In [17]:
import spacy

nlp = spacy.load('en', disable=['parser', 'ner'])

# NOUN, ADJ, VERB, ADV
def lemmatization(texts, allowed_postags=['NOUN']):
    texts_out = []
    for sent in texts:
#         print(sent)
        doc = nlp(" ".join(sent)) 
#         print(doc)
        output_text = [token.lemma_ for token in doc if token.pos_ in allowed_postags and token.is_stop == False]
#         print(output_text)
        if len(output_text) > 0:
            texts_out.append(output_text)
    return texts_out

In [18]:
clean_sents = lemmatization(clean_sents)

In [21]:
with open('clean_sents_adjusted.pkl', 'wb') as f:
    pickle.dump(clean_sents, f)

In [132]:
for t in clean_sents[0:5]:
    print(f'{" ".join(t)} \n')

transport connection want 

home gem 

bus pad 

location tenderloin_district 

city san_francisco location day airbnb head pillow 



In [133]:
id2word = gensim.corpora.Dictionary(clean_sents)

corpus = [id2word.doc2bow(t) for t in clean_sents]

In [136]:
import logging
import warnings
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train5 = gensim.models.ldamulticore.LdaMulticore(
                           corpus=corpus,
                           num_topics=10,
                           id2word=id2word)
#                            chunksize=100,
#                            workers=3, # Num. Processing Cores - 1
#                            passes=50,
#                            eval_every = 1,
#                            per_word_topics=True)
    
lda_train5.save('lda_train5.model')
lda_train5.load('lda_train5.model')

In [138]:
pprint(lda_train5.print_topics())

[(0,
  '0.181*"stay" + 0.058*"city" + 0.044*"family" + 0.037*"check" + 0.027*"way" '
  '+ 0.024*"friend" + 0.024*"time" + 0.021*"couple" + 0.019*"week" + '
  '0.017*"san_francisco"'),
 (1,
  '0.138*"home" + 0.043*"thing" + 0.037*"amenity" + 0.029*"guest" + '
  '0.017*"question" + 0.016*"hospitality" + 0.015*"suggestion" + 0.012*"suite" '
  '+ 0.012*"love" + 0.011*"et"'),
 (2,
  '0.168*"apartment" + 0.098*"space" + 0.061*"time" + 0.057*"parking" + '
  '0.034*"street" + 0.030*"spot" + 0.025*"car" + 0.024*"visit" + 0.019*"need" '
  '+ 0.016*"town"'),
 (3,
  '0.423*"place" + 0.064*"experience" + 0.050*"airbnb" + 0.028*"san_francisco" '
  '+ 0.022*"time" + 0.017*"san_fran" + 0.017*"unit" + 0.016*"city" + '
  '0.014*"dog" + 0.011*"photo"'),
 (4,
  '0.081*"bed" + 0.079*"day" + 0.048*"night" + 0.025*"lot" + 0.023*"person" + '
  '0.021*"picture" + 0.019*"tip" + 0.018*"noise" + 0.012*"michael" + '
  '0.012*"sleep"'),
 (5,
  '0.210*"location" + 0.184*"house" + 0.053*"people" + 0.039*"studio" + '


In [139]:
top_topics = lda_train5.get_document_topics(corpus[0])
top_topics.sort(key=lambda x: x[1], reverse=True)

print(top_topics)

[(8, 0.5248399), (2, 0.27515775), (4, 0.025000982), (0, 0.02500096), (5, 0.025000356), (1, 0.025000053), (9, 0.025000015), (7, 0.025000002), (3, 0.025), (6, 0.025)]


In [141]:
top_topics_2 = lda_train5.get_document_topics(corpus)

In [143]:
def AssignTopics(topics):
    topic_assign = np.zeros((101793987,10))
    for index, topic in enumerate (topics):
        for item in topic:
            if item[1] >= 0.15:
                topic_assign[index][item[0]] +=1
    return topic_assign

In [144]:
topic_matrix = AssignTopics(top_topics_2)

In [145]:
def AssignTopicsMatrix(topics):
    topic_assign = np.zeros((101793987,10))
    for index, topic in enumerate (topics):
        for item in topic:
            topic_assign[index][item[0]] = item[1]
    return topic_assign

In [146]:
topic_pro_matrix = AssignTopicsMatrix(top_topics_2)

In [148]:
topic_pro_matrix_df = pd.DataFrame(topic_pro_matrix)

In [149]:
topic_pro_matrix.shape

(101793987, 10)

Dataset is too larget to pickle, so I split it into two parts and then save them separately

In [151]:
topic_pro_matrix_df_part1 = topic_pro_matrix[:50000000]
topic_pro_matrix_df_part2 = topic_pro_matrix[50000001:101793987]

In [153]:
import pickle 

with open('topic_pro_matrix_df_part51.pkl', 'wb') as f:
    pickle.dump(topic_pro_matrix_df_part1, f)

In [154]:
import pickle 

with open('topic_pro_matrix_df_part52.pkl', 'wb') as f:
    pickle.dump(topic_pro_matrix_df_part2, f)